In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/root/data/transformers/model_zoo'

In [2]:
import numpy as np
import faiss
import torch
from tqdm import tqdm
from transformers import AlbertModel, BertTokenizer

In [20]:
class AlbertDocEncoder(object):
    def __init__(self, pretrained='voidful/albert_chinese_tiny'):
        self.tokenizer = BertTokenizer.from_pretrained(pretrained, mirror='tuna')
        self.model = AlbertModel.from_pretrained(pretrained, mirror='tuna')

    def encode_doc(self, doc):
        input_ids = self.tokenizer.encode(doc, add_special_tokens=False, return_tensors='pt')
        with torch.no_grad():
            outputs = self.model(input_ids, output_hidden_states=True)
        hidden_state = outputs.hidden_states
        vec = torch.mean(hidden_state[1] + hidden_state[-1], dim=1).squeeze()
        return vec


class FaissIndexer(object):
    def __init__(self, embed_dim, param='HNSW64', measure=faiss.METRIC_INNER_PRODUCT):
        self.index = faiss.index_factory(embed_dim, param, measure)  
    
    def build_index(self, vecs):
        self.index.add(vecs)

    def most_similar(self, query_vecs, top_k=10):
        dist, ind = self.index.search(query_vecs, k=top_k)
        return ind[0]


class WhiteTransform(object):
    def __init__(self, vecs, n_components=128):
        self.embed_in = vecs.shape[1]
        self.mu = self.moving_mean(vecs)
        self.cov = self.moving_cov(vecs)
        self.W = self.get_kernel(n_components)

    def moving_mean(self, vecs):
        mu = np.zeros((1, vecs.shape[1]), dtype=np.float32)
        for i in range(vecs.shape[0]):
            mu = i / (i + 1) * mu + 1 / (i + 1) * vecs[i, :]
        return mu
    
    def moving_cov(self, vecs):
        cov = np.zeros((vecs.shape[1], vecs.shape[1]), dtype=np.float32)
        for i in range(vecs.shape[0]):
            vec = vecs[i, :].reshape((1, -1))
            vec = vec - self.mu
            cur_cov = np.dot(vec.T, vec)
            cov = i / (i + 1) * cov + 1 / (i + 1) * cur_cov
        return cov
    
    def get_kernel(self, n_components):
        u, s, vh = np.linalg.svd(self.cov)
        W = np.dot(u, np.diag(1 / np.sqrt(s)))
        return W[:, :n_components]

    def transform_vecs(self, vecs):
        vecs = (vecs - self.mu).dot(self.W)
        return vecs / (vecs**2).sum(axis=1, keepdims=True) ** 0.5

In [4]:
pretrained = 'voidful/albert_chinese_tiny'
albert_enc = AlbertDocEncoder()

In [5]:
lines = open('../data/tnews/pretrain_data_5k.txt', 'r', encoding='utf8').read().splitlines()
print(len(lines))
lines[:3]

5000


['贷款资料真实承诺书',
 '特工被迫密裁战友，行刑前为战友暖心的做了四件事',
 '如果你是中学老师，可以给学生们放三部电影，你会怎么选择以充分达到教育的意义？']

In [6]:
n_sample = len(lines)
embed_dim = 312
embed_vecs = np.zeros((n_sample, embed_dim), dtype=np.float32)
for i in tqdm(range(len(lines))):
    vec = albert_enc.encode_doc(lines[i])
    embed_vecs[i, :] = vec

100%|██████████| 5000/5000 [01:00<00:00, 82.03it/s]


In [21]:
whiter = WhiteTransform(embed_vecs, n_components=128)

In [8]:
embed_vecs_white = whiter.transform_vecs(embed_vecs)

In [22]:
index_origin = FaissIndexer(embed_dim=embed_vecs.shape[1], param='HNSW64', measure=faiss.METRIC_L2)
index_origin.build_index(embed_vecs)

In [23]:
index_white = FaissIndexer(embed_dim=embed_vecs_white.shape[1], param='HNSW32', measure=faiss.METRIC_INNER_PRODUCT)
index_white.build_index(embed_vecs_white)

In [31]:
q_vec = albert_enc.encode_doc(u'比亚迪4月销量劲增20%').numpy()
q_vec = q_vec.reshape((1, -1))
for i in index_origin.most_similar(q_vec, 10):
    print(lines[i])

比特币半月暴跌近20%，这篇比特币买卖指南教你如何入场抄底
别克英朗4月销量公布同比暴跌32%，别克凯越即将复活救主
比特币共识会议召开，是否确定比特币涨势开启？5.9
如何评价“京东一季度整后每ADS收益0.71元，盘前跌近4%”？
新能源、燃油车双线飘红 比亚迪4月销量劲增20%
最前线｜连续两季度盈利的虎牙直播，IPO首日股价涨逾超30%
纯国际机票业务占比达30% 携程：其实我们做得挺苦的
马云放大招，余额宝不再限购限额，收益率高达4.4%
BA黑凤梨客场3比0战胜Hero久竞，暂时登顶东部榜首，你怎么看？
奔驰GLB来了比GLA大一圈，整体性能感很强，仅26万奥迪宝马没法比


In [32]:
q_vec = albert_enc.encode_doc(u'比亚迪4月销量劲增20%？').numpy()
q_vec = q_vec.reshape((1, -1))
q_vec_white = whiter.transform_vecs(q_vec)
for i in index_white.most_similar(q_vec_white, 10):
    print(lines[i])

比特今日——BTC
比特币半月暴跌近20%，这篇比特币买卖指南教你如何入场抄底
别克英朗4月销量公布同比暴跌32%，别克凯越即将复活救主
比亚迪宋智联尊贵版值得入手吗？
比特币共识会议召开，是否确定比特币涨势开启？5.9
新能源、燃油车双线飘红 比亚迪4月销量劲增20%
最前线｜连续两季度盈利的虎牙直播，IPO首日股价涨逾超30%
马云放大招，余额宝不再限购限额，收益率高达4.4%
如何评价“京东一季度整后每ADS收益0.71元，盘前跌近4%”？
补贴退坡影响业绩 比亚迪提应对措施
